# Goal

- We want to make a script that can run and webscrape from MAL from both the previous & airing season animes and webscrap information about the animes.
- Ideally, we want all interesting metrics pertaining to each anime, because the goal of this webscraper is to provide data for a future data visualization, which will help tell a story about each new season releases of anime.
- We want this script to be easily maintainable and re-usable

In [1]:
import os
from bs4 import BeautifulSoup
import requests
import time
from datetime import datetime
import json
from zipfile import ZipFile
import pandas as pd
import shutil
import zipfile
import random
import numpy as np
from jikanpy import Jikan

In [2]:
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException

In [3]:
# Gather links for the specific season & year of release anime you desire.
def Anime_data_links(name='tv'):
    
    """
    Takes in an input of either 'tv' or 'movie', to specify if you want to web scrape from either type of anime, default is
    set to 'tv'. This function will return a list of links for each anime in that category.
    
    """
    
    driver = webdriver.Chrome(executable_path=r'C:\chromedriver_win32/chromedriver.exe')
    wait = WebDriverWait(driver, 10) # Time our webdriver takes befiore timing out - stoping script if error during web scraping.

    anime_links = []
    for page in range(0,25000,50):
        time.sleep(1) # Time stop between each page, so not to overload server and risk an ip ban.
        driver.get(f'https://myanimelist.net/topanime.php?type={name}&limit=' + str(page))
    # Find all elements in page by CSS selector's element & save as a link variable
        link = driver.find_elements(By.CSS_SELECTOR, 'div[class="detail"] h3 a')
        for item in link:
            anime_links.append(item.get_attribute('href'))
            
    return anime_links


def seperate(lst,n):
    """
    Break our links into equal sized chunks, so its easier to run/complete.

    """
    for i in range(0,len(lst),n):
        yield lst[i:i + n]


def Anime_database(anime_links):
    
    """
    Takes a list of anime links and webscraps each individual anime for its title,genre,rating,url,source and studio.
    
    and then returns multiple lists of the webscraped elements.
    """

    driver = webdriver.Chrome(executable_path=r'C:\chromedriver_win32/chromedriver.exe')
    wait = WebDriverWait(driver, 10)

    titles = []
    ratings = []
    genres = []
    errors = []
    links = []
    studios = []
    sources = []

    for item_link in anime_links:
        time.sleep(1)
        driver.implicitly_wait(5) # Time driver should wait for when searching for an element not immediately present.
        driver.get(item_link)

        if requests.get(item_link).status_code == 404:
            continue


        # Some anime pages dont contain the genre element,so we must check, thats why we specified implicitly_wait above.
        elif driver.find_elements_by_xpath('//*[contains(text(), "Genre")]/parent::div'):
            url = item_link
            genre = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[contains(text(), "Genre")]/parent::div')))
            title = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div[class="h1-title"]')))
            rating = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div[class*="score-label"]')))
            studio = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[contains(text(), "Studios")]/parent::div')))
            source = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[contains(text(), "Source")]/parent::div')))  

            links.append(url)
            genres.append(genre.text)
            titles.append(title.text)
            ratings.append(rating.text)
            studios.append(studio.text)
            sources.append(source.text)

        else:
            name = driver.find_element_by_css_selector('div[class="h1-title"]')
            errors.append(name)
            continue
            
    return titles,ratings,genres,errors,links,studios,sources


In [ ]:
df = pd.DataFrame(
{
    'Url': links,
    'Title':titles,
    'Rating':ratings,
    'Genre':genres,
    'Studio': studios,
    'Source': sources
})

In [ ]:
df.to_csv('')